In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import os.path as path
import random
from IPython.display import HTML
from collections import Counter

import exdir
import numpy as np
import pandas as pd
from tqdm import tqdm
import skvideo
import yaml

os.chdir('/home/Gangus/cow-tus')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data_dir = 'data/single-instance-learning/temporal-downsample'

hypothesis_conditions = ['by-animal-number', 'hold-out-validation']
group_dir = path.join('data', 'split', *hypothesis_conditions)

split_counts = {
    'test': 15,
    'valid': 15,
    'train': -1
}

In [24]:
attrs_path = path.join(data_dir, 'attrs.csv')
attrs_df = pd.read_csv(attrs_path, index_col=2)


86

In [25]:
raw_attrs_df = pd.read_csv(path.join("/data/cow-tus-data/raw", 'labels.csv'))

In [30]:
set(raw_attrs_df['id']).symmetric_difference(attrs_df['exdir.exam_id'])

{'CH4815', 'WT4493', 'YW4426'}